In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [5]:
# take one copy for model development data
data = model_data.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [6]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [7]:
features_list = [
'yearOfBirth_mean',
'deviation_from_median_mean_age#last_180_Days',
'median_mean_quality_score#last_180_Days',
'statedIncome',
'count_of_perfios_done_applications#last_365_Days',
'avg_income#last_365_Days',
'pan_made_year_deviation_from_mean',
'deviation_from_mean_age/standard_deviation_age#last_365_Days',
'count_of_loan#last_180_Days',
'median_mean_loan_amount#last_30_Days',
'median_mean_line_score#last_30_Days',
'loan_amount_to_income#last_365_Days',
'count_of_credit_approved_applications#last_30_Days',
'age',
'deviation_from_median_mean_income#last_365_Days',
    
'month_year',
'G15_B24']

In [8]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3042246/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3042246/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 15), (13738, 15), (20606,), (13738,))

In [11]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 15) (4122, 15) (13738, 15)


In [12]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [13]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
     'max_depth': 1,
    'subsample': 0.2,
    'colsample_bytree': 0.2,
    'lambda': 6.4,
    'alpha': 4.0,
    'gamma': 5.6,
    'scale_pos_weight': 1,
    'random_state': 24,
}
evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 90,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.50000	eval-auc:0.50000
[10]	train-auc:0.52195	eval-auc:0.50197
[20]	train-auc:0.55569	eval-auc:0.53449
[30]	train-auc:0.55670	eval-auc:0.52384
[40]	train-auc:0.55622	eval-auc:0.54853
[50]	train-auc:0.56260	eval-auc:0.55863
[60]	train-auc:0.55258	eval-auc:0.56177
[70]	train-auc:0.55134	eval-auc:0.56871
[80]	train-auc:0.55992	eval-auc:0.56959
[89]	train-auc:0.55992	eval-auc:0.56959


In [14]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.56, Gini: 0.12, KS: 0.09
Eval AUC: 0.57, Gini: 0.14, KS: 0.12
Test AUC: 0.55, Gini: 0.10, KS: 0.08
OOT AUC: 0.52, Gini: 0.05, KS: 0.05


In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.084042         0.078875        0.096055        0.075228   1059   
9          0.090056         0.073453        0.075213        0.071908   1066   
8          0.078278         0.070721        0.071898        0.069701   1022   
7          0.077130         0.068793        0.069697        0.067709   1115   
6          0.071618         0.067275        0.067692        0.066406   1131   
5          0.073516         0.065557        0.066355        0.065065   1061   
4          0.075249         0.064422        0.065055        0.063282   1103   
3          0.070717         0.062949        0.063266        0.062674   1004   
2          0.074949         0.061460        0.062664        0.060726    974   
1          0.067088         0.059061        0.060705        0.054964   1267   

        label_good  label_bad  
Decile                         
10             970         89  
9              970         96  
8              942         80  
7             1029         86  
6             1050         81  
5              983         78  
4             1020         83  
3              933         71  
2              901         73  
1             1182         85

In [16]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.095209         0.076319        0.093978        0.072716   1649   
9          0.087485         0.070888        0.072683        0.069612   1646   
8          0.062222         0.068345        0.069601        0.067375   1575   
7          0.069808         0.066550        0.067372        0.065468   1719   
6          0.067354         0.065016        0.065460        0.064414   1648   
5          0.062820         0.063462        0.064387        0.063031   1369   
4          0.051242         0.062375        0.063026        0.061226   1932   
3          0.045549         0.060946        0.061213        0.060726   1449   
2          0.061321         0.059620        0.060705        0.058689   1696   
1          0.052193         0.057408        0.058661        0.054964   1801   

        label_good  label_bad  
Decile                         
10            1492        157  
9             1502        144  
8             1477         98  
7             1599        120  
6             1537        111  
5             1283         86  
4             1833         99  
3             1383         66  
2             1592        104  
1             1707         94

In [17]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.080645         0.076447        0.091958        0.072726   1364   
9          0.074422         0.070931        0.072718        0.069633   1384   
8          0.063830         0.068275        0.069625        0.067372   1363   
7          0.070202         0.066485        0.067347        0.065460   1339   
6          0.061429         0.064957        0.065428        0.064117   1400   
5          0.060562         0.063344        0.064075        0.063010   1387   
4          0.046282         0.062225        0.062998        0.061171   1318   
3          0.061181         0.060904        0.061159        0.060678   1422   
2          0.050251         0.059478        0.060672        0.058689   1194   
1          0.044671         0.057414        0.058661        0.054964   1567   

        label_good  label_bad  
Decile                         
10            1254        110  
9             1281        103  
8             1276         87  
7             1245         94  
6             1314         86  
5             1303         84  
4             1257         61  
3             1335         87  
2             1134         60  
1             1497         70

In [18]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
0                                   yearOfBirth_mean         3.0
5  deviation_from_mean_age/standard_deviation_age...         3.0
1       deviation_from_median_mean_age#last_180_Days         2.0
8    deviation_from_median_mean_income#last_365_Days         2.0
2            median_mean_quality_score#last_180_Days         2.0
4                  pan_made_year_deviation_from_mean         1.0
3                           avg_income#last_365_Days         1.0
6               median_mean_loan_amount#last_30_Days         1.0
7                                                age         1.0


In [19]:
feature_importance_df

Feature  Importance
0                                   yearOfBirth_mean         3.0
5  deviation_from_mean_age/standard_deviation_age...         3.0
1       deviation_from_median_mean_age#last_180_Days         2.0
8    deviation_from_median_mean_income#last_365_Days         2.0
2            median_mean_quality_score#last_180_Days         2.0
4                  pan_made_year_deviation_from_mean         1.0
3                           avg_income#last_365_Days         1.0
6               median_mean_loan_amount#last_30_Days         1.0
7                                                age         1.0

In [20]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_LA_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_LA_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_LA_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_LA_Oot_Pred_1524.csv',index=False)


In [21]:
#Save the model
xgb_model.save_model('LeoPayu_LA_15_in_24mob.json')
print('Model Saved')

Model Saved
